# **TensorFlow Logistic Regression**

In [14]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np

In [15]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_features=784

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32) # Convert to float32
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.  # Normalize images from [0,255] to [0,1]


**Setting up Hyperparameters**

In [16]:
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [17]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)  # Shuffling and batching

**Initializing weights and biases**

In [18]:
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

**Logistic Regression and cost function**

In [19]:
def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)
def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))


**Accuracy Metrics**

In [20]:
def accuracy(y_pred, y_true):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.optimizers.SGD(learning_rate)

**Running Optimizations**

In [21]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    gradients = g.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [22]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 1087.864746, accuracy: 0.738281
step: 100, loss: 638.513428, accuracy: 0.808594
step: 150, loss: 103.019150, accuracy: 0.890625
step: 200, loss: 69.949799, accuracy: 0.921875
step: 250, loss: 71.271210, accuracy: 0.945312
step: 300, loss: 100.956909, accuracy: 0.875000
step: 350, loss: 121.919418, accuracy: 0.902344
step: 400, loss: 139.982834, accuracy: 0.859375
step: 450, loss: 61.643944, accuracy: 0.941406
step: 500, loss: 110.057816, accuracy: 0.890625
step: 550, loss: 55.506897, accuracy: 0.937500
step: 600, loss: 60.239090, accuracy: 0.925781
step: 650, loss: 38.048409, accuracy: 0.953125
step: 700, loss: 60.614609, accuracy: 0.933594
step: 750, loss: 81.791809, accuracy: 0.910156
step: 800, loss: 54.510468, accuracy: 0.945312
step: 850, loss: 61.915745, accuracy: 0.949219
step: 900, loss: 74.214638, accuracy: 0.925781
step: 950, loss: 31.731524, accuracy: 0.960938
step: 1000, loss: 59.634373, accuracy: 0.941406


**Testing Model Accuracy**

In [23]:
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.914300
